# Tier 1: Risk-Neutral Valuation

**Learning Objectives:**
- Understand why we use risk-neutral pricing
- See the impact of using wrong drift
- Learn to verify pricing implementations

**Key Concept [T1]:**
Under the risk-neutral measure Q, all tradeable assets have expected return r (risk-free rate).

**Duration:** ~20 minutes

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

from annuity_pricing.options.pricing.black_scholes import black_scholes_call

## 1. The Two Measures

| Measure | Symbol | Drift | Use Case |
|---------|--------|-------|----------|
| Physical | P | μ (historical) | Forecasting |
| Risk-Neutral | Q | r - q | Pricing |

### Why Does It Matter?

If you use the **wrong drift** in Monte Carlo pricing:
- You get the wrong price
- Your hedges will be wrong
- Put-call parity won't hold

In [ ]:
# Parameters
S = 100.0
K = 100.0
r = 0.05       # Risk-free rate
q = 0.02       # Dividend yield
sigma = 0.20
T = 1.0

# Historical vs risk-neutral
mu_historical = 0.10  # Historical equity return
mu_risk_neutral = r - q  # Risk-neutral drift

print(f"Historical drift (μ): {mu_historical:.1%}")
print(f"Risk-neutral drift (r-q): {mu_risk_neutral:.1%}")
print(f"Difference: {abs(mu_historical - mu_risk_neutral):.1%}")

## 2. Monte Carlo with Wrong vs Right Drift

In [ ]:
def monte_carlo_call(S, K, drift, sigma, T, r_discount, n_paths=100_000, seed=42):
    """Price call using Monte Carlo with specified drift."""
    rng = np.random.default_rng(seed)
    
    # Simulate terminal prices (exact GBM)
    Z = rng.standard_normal(n_paths)
    S_T = S * np.exp((drift - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    
    # Payoffs
    payoffs = np.maximum(S_T - K, 0)
    
    # Discounted expected value
    price = np.exp(-r_discount * T) * payoffs.mean()
    se = np.exp(-r_discount * T) * payoffs.std() / np.sqrt(n_paths)
    
    return price, se

In [ ]:
# Black-Scholes reference
bs_price = black_scholes_call(S, K, r, q, sigma, T)

# MC with WRONG drift (historical)
mc_wrong, se_wrong = monte_carlo_call(S, K, mu_historical, sigma, T, r)

# MC with CORRECT drift (risk-neutral)
mc_correct, se_correct = monte_carlo_call(S, K, mu_risk_neutral, sigma, T, r)

print("Results:")
print(f"  Black-Scholes:     ${bs_price:.4f}")
print(f"  MC (wrong drift):  ${mc_wrong:.4f} ± ${se_wrong:.4f}")
print(f"  MC (correct drift): ${mc_correct:.4f} ± ${se_correct:.4f}")
print()
print(f"Error (wrong):   ${abs(mc_wrong - bs_price):.4f} ({abs(mc_wrong - bs_price)/bs_price:.1%})")
print(f"Error (correct): ${abs(mc_correct - bs_price):.4f} ({abs(mc_correct - bs_price)/bs_price:.1%})")

## 3. Visualizing the Impact

In [ ]:
# Different historical drifts
drifts = np.linspace(-0.05, 0.20, 50)
mc_prices = [monte_carlo_call(S, K, d, sigma, T, r, n_paths=10_000)[0] for d in drifts]

plt.figure(figsize=(10, 5))
plt.plot(drifts * 100, mc_prices, 'b-', linewidth=2, label='MC Price')
plt.axhline(y=bs_price, color='green', linestyle='--', linewidth=2, label=f'BS Price (${bs_price:.2f})')
plt.axvline(x=mu_risk_neutral * 100, color='red', linestyle=':', linewidth=2, label=f'Correct drift ({mu_risk_neutral:.1%})')
plt.xlabel('Drift (%)')
plt.ylabel('Call Price ($)')
plt.title('Impact of Drift on Option Price')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Key Insight:")
print("  Only at the RISK-NEUTRAL drift (r-q) does MC match BS.")
print("  Using historical drift gives wrong prices!")

## 4. Key Takeaways

1. **Risk-neutral drift = r - q**, not historical returns
2. **Option prices are arbitrage-based**, not expectation-based
3. **MC must converge to BS** for vanilla options (both use same framework)
4. **Test your implementation** against BS before using for exotics

## Next

Continue to `02_put_call_parity.ipynb` to learn the fundamental arbitrage relationship.